Per entrar a una sessió de Spark i iniciar un builder en DeltaLake

In [1]:
#pip install pyspark
#pip install delta-spark

import pyspark
from delta import *

#!wget -O "HR_comma_sep.csv" "https://mydisk.cs.upc.edu/s/3o33yciBHADiFCD/download/HR_comma_sep.csv"

builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

24/04/23 08:44:20 WARN Utils: Your hostname, Ainas-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.33 instead (on interface en0)
24/04/23 08:44:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/aina/.ivy2/cache
The jars for the packages stored in: /Users/aina/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-151186ec-834d-496c-a46f-b34a20edebde;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 188ms :: artifacts dl 7ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.1.0 from central in [default]
	io.delta#delta-storage;3.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   

## Comandes com si estessis a una sessió de Spark

Llegir els arxius i guardar-los en format DeltaLake, que ens garantitza que es compleixen les restriccions ACID, i dona avanatatges 




Carregar i llegir els 3 arxius de tenim

In [3]:
#Arxiu Parquet 
sales = spark.read.parquet("./datalake/sales_data/2024-04-22_SuperstoreSalesTraining.parquet")
sales.show()


24/04/23 08:45:19 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---+--------------+----------+-----+--------+----------+--------------+--------+-------+-------------+-------------------+---------------+----------+--------------------+--------------------+----------------+-----------+--------------------+-------------+------------------+--------------------+-------------+-----------+----------+--------------+--------------------+
|Row|Order Priority|Order Date|Order|Discount|Unit Price|Order Quantity|   Sales| Profit|Shipping Cost|Product Base Margin|     Department| Container|            Category|                Item|Customer Segment|Customer_ID|       Customer Name|       Region|             State|    Country / Region|         City|Postal Code| Ship Date|     Ship Mode|           SubRegion|
+---+--------------+----------+-----+--------+----------+--------------+--------+-------+-------------+-------------------+---------------+----------+--------------------+--------------------+----------------+-----------+--------------------+-------------+----

In [ ]:
from pyspark.sql.functions import col

# Lista de todos los caracteres inválidos que quieres reemplazar o eliminar
invalid_chars = [' ', ';', '{', '}', '(', ')', '\n', '\t', '=']

# Función para limpiar los nombres de las columnas reemplazando los caracteres no válidos
def clean_column_name(column_name):
    for invalid_char in invalid_chars:
        column_name = column_name.replace(invalid_char, "_")  # Reemplaza por subrayado o cualquier otro caracter válido que prefieras
    return column_name

# Aplicar la función de limpieza a cada columna
cleaned_sales = sales.select([col(c).alias(clean_column_name(c)) for c in sales.columns])

# Ahora guarda el DataFrame limpio en formato Delta
cleaned_sales.write.mode("overwrite").format("delta").save("./deltalake/sales_data/")


In [15]:
from pyspark.sql.functions import col

# filtrar EEUU
sales_usa = sales.filter(col("Country / Region") == "United States of America")

# eliminar row
sales_usa = sales_usa.dropDuplicates(subset=[col for col in sales_usa.columns if col != "row"])

# eliminar customer_name
sales_usa = sales_usa.drop("Customer Name")

# no fa falta fer imputació de missings perque quan filtrem per USA no ens queden columnes amb missings

# eliminar missings a postal_code
sales_usa = sales_usa.dropna(subset=["Postal Code"])

# eliminar missings a subregions
sales_usa = sales_usa.dropna(subset=["SubRegion"])

# eliminar files que missing a totes les columnes
sales_usa = sales_usa.dropna(how="all")

# Finalmente, para guardar el DataFrame procesado, puedes usar:
# sales_usa.write.csv("ruta/a/donde/guardar/resultados", header=True)

sales_usa.printSchema()


root
 |-- Row: long (nullable = true)
 |-- Order Priority: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Order: long (nullable = true)
 |-- Discount: string (nullable = true)
 |-- Unit Price: double (nullable = true)
 |-- Order Quantity: long (nullable = true)
 |-- Sales: double (nullable = true)
 |-- Profit: double (nullable = true)
 |-- Shipping Cost: double (nullable = true)
 |-- Product Base Margin: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- Container: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Item: string (nullable = true)
 |-- Customer Segment: string (nullable = true)
 |-- Customer_ID: long (nullable = true)
 |-- Region: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Country / Region: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Postal Code: double (nullable = true)
 |-- Ship Date: string (nullable = true)
 |-- Ship Mode: string (nullable = true)
 |-- SubRe

In [ ]:
# finalitzar sessió de Spark
spark.stop()